# Business Problem

### Problem Statement

The Business problem is to consider a customer who want to open a Frozen Yogurt Shop in specific area

### Requirements

The customer require to predict the profits befor thier invest into this business

### Background

Planing ahead and investiment prediction is major problem in today's AI world. Customers require to understand the market and have a clear prediction about thier business even before any investiment.
Using the current available data on how other business in the same business domain and in the same area will clarify how the current business statement and how it will be in future

### Data description

By fetching data related to the current business in the same domain as of our example Frozen Yogurt Shop resturant, and also in the same area. 
The data we need is:
- how many frozen yogert/resturants in this city
- how many customers per day going into these resturants
- how many going in weekends
- try fetch data for an entir year
- tipping is usually percentage of total billing, so if we could get the tipping of the current shops, we can understand how many these resturants do per week

### Get Number and Names of Frozen Yogurt Shop in Austin Texas

In [1]:
"""Initialize Library"""
import requests
import pandas as pd
import numpy as np
from lxml import etree
from geopy.geocoders import Nominatim

from opencage.geocoder import OpenCageGeocode
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import json

print('Libraries imported.')

Libraries imported.


In [2]:
"""Define Foursquare Credentials and Version"""
CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

key = '...'
geocoder = OpenCageGeocode(key)

# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [18]:
address = 'Austin, Texas'
results = geocoder.geocode(address)

austin_latitude = results[0]['geometry']['lat']
austin_longitude = results[0]['geometry']['lng']
print(austin_latitude, austin_latitude)

30.2711286 30.2711286


Get venue category

In [5]:
cat_url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID,CLIENT_SECRET,VERSION)
r = requests.get(url= cat_url)

In [6]:
def parse_categories(cat, venue):
    if cat['name'] == venue:
        return True, cat['id'], ' '
    if(len(cat['categories']) > 0):
        for c_cat in cat['categories']:
            found, c_id, p_id = parse_categories(c_cat, venue)
            if(found):
                return found, c_id, c_cat['id']
    return False, ' ', ' '

In [7]:
fro_yogrt_cat = ''
parent_cat = ''
found = False
c_id = ''
p_id = ''
#parse_cat = ''
if r.ok:
    cat_response = json.loads(r.text)
    for p_cat in cat_response['response']['categories']:
        found, c_id, p_id  = parse_categories(p_cat, 'Frozen Yogurt Shop')
        if(found):
            break
if found:
    fro_yogrt_cat = c_id
    parent_cat = p_id
fro_yogrt_cat, parent_cat

('512e7cae91d4cbb4e5efe0af', '4bf58dd8d48988d1d0941735')

It is found that, Frozen Yogurt is sub-category of Dessert category
The plan now is to get:  
- All frozen yogurt shops  
- All Dessert shops 

In [32]:
def getFroDF(fro_response, defaultCity = 'unknown'):
    id_list = []
    name_list = []
    city_list = []
    lat_list = []
    lng_list = []
    for venue in fro_response['response']['venues']:
        id_list.append(venue['id'])
        name_list.append(venue['name'])
        if('city' in venue['location'].keys()):
            city_list.append(venue['location']['city'])
        else:
            city_list.append(defaultCity)
        lat_list.append(venue['location']['lat'])
        lng_list.append(venue['location']['lng'])
    return id_list, name_list, city_list, lat_list, lng_list

In [19]:
fro_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&near={}&categoryId={}'.format(
CLIENT_ID,CLIENT_SECRET,VERSION, address, fro_yogrt_cat)
r = requests.get(url= fro_url)
fro_df = pd.DataFrame(columns=['ven_id', 'ven_name', 'ven_city', 'ven_lat', 'ven_lng'])
if r.ok:
    fro_response = json.loads(r.text)
    id_list, name_list, city_list, lat_list, lng_list = getFroDF(fro_response)
    fro_df = pd.DataFrame({'ven_id': id_list, 'ven_name': name_list, 'ven_city': city_list, 'ven_lat': lat_list, 
                           'ven_lng': lng_list})
fro_df.to_csv('fro_df.csv')
fro_df.head()

,ven_city,ven_id,ven_lat,ven_lng,ven_name
0,Austin,4b7cc2dff964a52078a42fe3,30.281256,-97.740738,TCBY
1,Buda,5c71cefddee770002c369982,30.082615,-97.823440,Freddy’s Frozen Custard & Steakburgers
2,Austin,58bc85a151d19e7f8b44ad9b,30.394837,-97.749304,Menchie's at Great Hills
3,Austin,553ce4f7498e9ba5843cb7e9,30.299627,-97.721174,Menchie's Frozen Yogurt
4,Pflugerville,57afc76d498e1ecd88d497b5,30.461910,-97.598716,Menchie's


In [20]:
fro_df = pd.read_csv('fro_df.csv', index_col=0)
fro_df.head()

,ven_city,ven_id,ven_lat,ven_lng,ven_name
0,Austin,4b7cc2dff964a52078a42fe3,30.281256,-97.740738,TCBY
1,Buda,5c71cefddee770002c369982,30.082615,-97.823440,Freddy’s Frozen Custard & Steakburgers
2,Austin,58bc85a151d19e7f8b44ad9b,30.394837,-97.749304,Menchie's at Great Hills
3,Austin,553ce4f7498e9ba5843cb7e9,30.299627,-97.721174,Menchie's Frozen Yogurt
4,Pflugerville,57afc76d498e1ecd88d497b5,30.461910,-97.598716,Menchie's


In [11]:
fro_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&near={}&categoryId={}'.format(
CLIENT_ID,CLIENT_SECRET,VERSION, address, parent_cat)
r = requests.get(url= fro_url)
dessert_df = pd.DataFrame(columns=['ven_id', 'ven_name', 'ven_city', 'ven_lat', 'ven_lng'])
if r.ok:
    fro_response = json.loads(r.text)
    id_list, name_list, city_list, lat_list, lng_list = getFroDF(fro_response)
    dessert_df = pd.DataFrame({'ven_id': id_list, 'ven_name': name_list, 'ven_city': city_list, 'ven_lat': lat_list, 
                           'ven_lng': lng_list})
dessert_df.to_csv('dessert_df.csv')
dessert_df.head()

,ven_city,ven_id,ven_lat,ven_lng,ven_name
0,Austin,4d780e87ece5a1cddcd58fa4,30.264501,-97.744821,Berry Austin
1,Austin,5af79581f8cbd400394cf2a3,30.267235,-97.752750,The Baked Bear
2,Austin,4e9cb24446904f8f933fc63b,30.255654,-97.762526,Lick Ice Creams
3,Buda,5c71cefddee770002c369982,30.082615,-97.823440,Freddy’s Frozen Custard & Steakburgers
4,Austin,4b25904af964a520db7324e3,30.391411,-97.713988,Culver's


In [12]:
dessert_df = pd.read_csv('dessert_df.csv', index_col=0)
dessert_df.head()

,ven_city,ven_id,ven_lat,ven_lng,ven_name
0,Austin,4d780e87ece5a1cddcd58fa4,30.264501,-97.744821,Berry Austin
1,Austin,5af79581f8cbd400394cf2a3,30.267235,-97.752750,The Baked Bear
2,Austin,4e9cb24446904f8f933fc63b,30.255654,-97.762526,Lick Ice Creams
3,Buda,5c71cefddee770002c369982,30.082615,-97.823440,Freddy’s Frozen Custard & Steakburgers
4,Austin,4b25904af964a520db7324e3,30.391411,-97.713988,Culver's


### Let's plot that into the map

In [41]:
# create map of Austin Frozen yogurt using latitude and longitude values
map_fro_austin = folium.Map(location=[austin_latitude, austin_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(fro_df['ven_lat'], fro_df['ven_lng'], fro_df['ven_name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc316c',
        fill_opacity=0.7,
        parse_html=False).add_to(map_fro_austin)

for lat, lng, label in zip(dessert_df['ven_lat'], dessert_df['ven_lng'], dessert_df['ven_name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#31cc55',
        fill_opacity=0.7,
        parse_html=False).add_to(map_fro_austin)
    
map_fro_austin

### Now, let's compare it with another city like Paris

In [39]:
paris_address = 'Paris, France'
results = geocoder.geocode(paris_address)

paris_latitude = results[0]['geometry']['lat']
paris_longitude = results[0]['geometry']['lng']
print(paris_latitude, paris_longitude)

48.8566101 2.3514992


In [33]:
paris_fro_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&near={}&categoryId={}'.format(
CLIENT_ID,CLIENT_SECRET,VERSION, paris_address, fro_yogrt_cat)
pr = requests.get(url= paris_fro_url)
paris_fro_df = pd.DataFrame(columns=['ven_id', 'ven_name', 'ven_city', 'ven_lat', 'ven_lng'])
if pr.ok:
    paris_fro_response = json.loads(pr.text)
    id_list, name_list, city_list, lat_list, lng_list = getFroDF(paris_fro_response, 'Paris')
    paris_fro_df = pd.DataFrame({'ven_id': id_list, 'ven_name': name_list, 'ven_city': city_list, 'ven_lat': lat_list, 
                           'ven_lng': lng_list})
paris_fro_df.to_csv('paris_fro_df.csv')
paris_fro_df.head()

,ven_city,ven_id,ven_lat,ven_lng,ven_name
0,Paris,4ff07da4e4b0b61f2a827eb3,48.859083,2.353587,Yogurt Factory
1,Paris,5acf7c0fe1f2284dc91e44f7,48.853889,2.337537,California Bliss
2,Paris,535bc50f11d260e3a8777a7a,48.861782,2.351140,l'atelier des glaces
3,Paris,539563e9498ecef183248e49,48.852635,2.335917,Dip & Go
4,Paris,57cac9cd498e41a4a5d021cb,48.861485,2.352987,Yogurt Factory


In [34]:
paris_parent_fro_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&near={}&categoryId={}'.format(
CLIENT_ID,CLIENT_SECRET,VERSION, paris_address, parent_cat)
ppr = requests.get(url= paris_parent_fro_url)
paris_parent_fro_df = pd.DataFrame(columns=['ven_id', 'ven_name', 'ven_city', 'ven_lat', 'ven_lng'])
if r.ok:
    paris_parent_fro_response = json.loads(ppr.text)
    id_list, name_list, city_list, lat_list, lng_list = getFroDF(paris_parent_fro_response)
    paris_parent_fro_df = pd.DataFrame({'ven_id': id_list, 'ven_name': name_list, 'ven_city': city_list, 
                                        'ven_lat': lat_list, 'ven_lng': lng_list})
paris_parent_fro_df.to_csv('paris_parent_fro_df.csv')
paris_parent_fro_df.head()

,ven_city,ven_id,ven_lat,ven_lng,ven_name
0,Paris,5b81a899249623002cbde2b3,48.854110,2.349764,Amorino
1,Paris,4bc5e23151b376b0ce8e1a6f,48.871036,2.303276,Ladurée
2,Paris,4b1d1a75f964a520e00b24e3,48.866578,2.325459,Pierre Hermé
3,Paris,5a3e6731b04056376864ad46,48.871514,2.303698,86 Champs (L'Occitane x Pierre Hermé)
4,Paris,4adcda14f964a5203a3721e3,48.863438,2.287255,Carette


In [40]:
# create map of Paris Frozen yogurt using latitude and longitude values
map_fro_paris = folium.Map(location=[paris_latitude, paris_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(paris_fro_df['ven_lat'], paris_fro_df['ven_lng'], paris_fro_df['ven_name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc316c',
        fill_opacity=0.7,
        parse_html=False).add_to(map_fro_paris)

for lat, lng, label in zip(paris_parent_fro_df['ven_lat'], paris_parent_fro_df['ven_lng'], paris_parent_fro_df['ven_name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#31cc55',
        fill_opacity=0.7,
        parse_html=False).add_to(map_fro_paris)
    
map_fro_paris

## Observation/Conclusion

As it can be noted by compare between the Austin and Paris maps, that is clear the frozen yogurt shops are concentrated in the cities downtown and very little fare away from the cities downtowns.

### Recommendation

My recommendation to the customer that the best area to open a forzen yogurt shop should be around the cities downtown not far away from that.